In [2]:
from ris import pysqldb
from IPython.display import clear_output
from collections import defaultdict
from collections import defaultdict, namedtuple
import datetime 
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

clear_output()
timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
ts = datetime.datetime.now().strftime('%m-%d-%Y')

print 'Notebook run: {}'.format(timestamp)
print os.getcwd()

Notebook run: 2020-10-19_12:58
E:\RIS\Staff Folders\Samuel\Requests\SH\Crash_Request_30320


In [3]:
#Database connection
sqldb = pysqldb.DbConnect(server='dot55sql01', database='forms', type='MS', user = 'arcgis', password = 'arcgis')

	Missing SQL Server Native Client 10.0                     datetime2 will not be interpreted correctly

Database connection (MS) to forms on dot55sql01 - user: arcgis 
Connection established 2020-10-19 12:58:12
- Query run 2020-10-19 12:58:12.788000
 Query time: Query run in 103000 microseconds 
 * Returned 0 rows *
	Missing SQL Server Native Client 10.0                     datetime2 will not be interpreted correctly



### Reading in Crash Request Nodeids 

In [3]:
#cr_nodes = pd.read_excel('E:\RIS\Staff Folders\Samuel\Requests\SH\Crash_Request_30320\crash request 2-28-2020.xlsx')

In [4]:
#cr_nodes = pd.read_excel('file:///E:\RIS\Staff%20Folders\Samuel\Requests\SH\Crash_Request_30320\crash%20request%203-16-2020.xlsx')
#cr_nodes.columns
#nodes = tuple(map(int,(cr_nodes.NODEID)))

In [5]:
nodes = '0036772'

In [6]:
sqldb.connect()

Database connection (MS) to forms on dot55sql01 - user: arcgis 
Connection established 2020-09-10 11:48:48


In [7]:
sqldb.query("""
SELECT masterid
FROM OPENQUERY(dotgissql01, 
'SELECT distinct masterid FROM [CLION].[dbo].[node] 
WHERE nodeid in ({nodes})') l 
""".format(nodes=nodes))
masters = [i[0] for i in sqldb.data]


- Query run 2020-09-10 11:48:50.092000
 Query time: Query run in 352000 microseconds 
 * Returned 1 rows *


In [8]:
masters

[36772]

### All NYPD Crash information Related to Crash Request Nodes from 2017 to Present

In [9]:
sqldb.connect()

Database connection (MS) to forms on dot55sql01 - user: arcgis 
Connection established 2020-09-10 11:48:52


In [10]:
data1 = sqldb.dfquery("""

SELECT DISTINCT coalesce(core.nodeid, core.lion_node_number) NODEID
               ,core.INTEGRATION_ID
               ,core.ACCIDENT_DT RECORD_DATE
               ,left(stuff(STUFF(right('000000'+cast(core.ACCIDENT_TIME_WID AS VARCHAR),6),5,0,':'),3,0,':'), 5) RECORD_TIME
               ,core.SRC_ADDRESS_TYPE ADDRESS_TYPE
               ,core.SRC_ON_STREET MAIN_STREET
               ,core.SRC_CROSS_STREET CROSS_STREET
               ,core.ACCIDENT_DIAGRAM COLLISION_TYPE
               ,core.TRAFFIC_CONTROL
               ,sum(CASE WHEN vic.PED_NONPED = 'Occupant' and vic.INJ_KILLED = 'Injured' THEN 1 ELSE 0 END) MVO
               ,sum(CASE WHEN vic.PED_NONPED = 'Bicyclist' and vic.INJ_KILLED = 'Injured' THEN 1 ELSE 0 END) BI
               ,sum(CASE WHEN vic.PED_NONPED = 'Pedestrian' and vic.INJ_KILLED = 'Injured' THEN 1 ELSE 0 END) PED
               ,sum(CASE WHEN vic.INJ_KILLED = 'Injured' THEN 1 ELSE 0 END) TOTAL_INJURIES
               ,sum(CASE WHEN vic.PED_ACTION = 'Crossing With Signal' THEN 1 ELSE 0 END) PED_ACTION_CROSSING_WITH_SIGNAL
               ,sum(CASE WHEN vic.PED_ACTION = 'Crossing Against Signal' THEN 1 ELSE 0 END) PED_ACTION_CROSSING_AGAINST_SIGNAL
               ,sum(CASE WHEN vic.PED_ACTION in ('Crossing, No Signal, or Crosswalk', 'Crossing, No Signal, Marked Crosswalk') THEN 1 ELSE 0 END) PED_ACTION_Crossing_NO_SIGNAL
               ,sum(CASE WHEN vic.PED_ACTION not in ('Crossing With Signal', 'Crossing Against Signal', 'Crossing, No Signal, or Crosswalk', 'Crossing, No Signal, Marked Crosswalk') 
                         THEN 1 ELSE 0 END) PED_ACTION_OTHER
               ,core.ACCIDENT_DESC ACCIDENT_DESCRIPTION
FROM [FORMS].[dbo].[WC_ACCIDENT_F] core
LEFT JOIN (SELECT DISTINCT ACCIDENT_ID
                          ,PED_ACTION
                          ,PED_NONPED
                          ,PERSON_ROLE_CODE
                          ,INJ_KILLED
           FROM [FORMS].[dbo].[WC_ACCIDENT_VICTIM_F]
           WHERE PERSON_ROLE_CODE in ('Driver','Passenger','Pedestrian','In-Line Skater')) vic
ON core.integration_id = vic.accident_id
WHERE YEAR(core.ACCIDENT_DT)>2016
AND coalesce(core.VOID_STATUS_CD , 'N') ='N'
AND (core.nodeid in ({nodes}) or core.masterid in ({masters}))
AND coalesce(core.nodeid, core.lion_node_number) is not null
GROUP BY coalesce(core.nodeid, core.lion_node_number) 
        ,core.INTEGRATION_ID
        ,core.ACCIDENT_DT 
        ,left(stuff(STUFF(right('000000'+cast(core.ACCIDENT_TIME_WID AS VARCHAR),6),5,0,':'),3,0,':'), 5) 
        ,core.SRC_ADDRESS_TYPE 
        ,core.SRC_ON_STREET 
        ,core.SRC_CROSS_STREET 
        ,core.ACCIDENT_DIAGRAM
        ,core.TRAFFIC_CONTROL
        ,core.ACCIDENT_DESC 
        
        
""".format(nodes=nodes, masters= str(masters)[1:-1]))

### Writing data to Excel Sheet 

In [12]:
#data.to_excel('crash_request_output_{}.xlsx'.format(ts.replace('-','')),index=False)

In [13]:
writer = pd.ExcelWriter('bx_crash_request_output_{}.xlsx'.format(ts.replace('-','')), engine='xlsxwriter')    
data1.to_excel(writer, sheet_name='Sheet1', index= False)
writer.save() 

## Testing

In [4]:
data = sqldb.dfquery("""

SELECT DISTINCT coalesce(core.nodeid, core.lion_node_number) NODEID
               ,core.INTEGRATION_ID
               ,core.ACCIDENT_DT RECORD_DATE
               ,left(stuff(STUFF(right('000000'+cast(core.ACCIDENT_TIME_WID AS VARCHAR),6),5,0,':'),3,0,':'), 5) RECORD_TIME
               ,core.SRC_ADDRESS_TYPE ADDRESS_TYPE
               ,core.SRC_ON_STREET MAIN_STREET
               ,core.SRC_CROSS_STREET CROSS_STREET
               ,core.ACCIDENT_DIAGRAM COLLISION_TYPE
               ,core.TRAFFIC_CONTROL
               ,sum(CASE WHEN vic.PED_NONPED = 'Occupant' and vic.INJ_KILLED = 'Injured' THEN 1 ELSE 0 END) MVO
               ,sum(CASE WHEN vic.PED_NONPED = 'Bicyclist' and vic.INJ_KILLED = 'Injured' THEN 1 ELSE 0 END) BI
               ,sum(CASE WHEN vic.PED_NONPED = 'Pedestrian' and vic.INJ_KILLED = 'Injured' THEN 1 ELSE 0 END) PED
               ,sum(CASE WHEN vic.INJ_KILLED = 'Injured' THEN 1 ELSE 0 END) TOTAL_INJURIES
               ,sum(CASE WHEN vic.PED_ACTION = 'Crossing With Signal' THEN 1 ELSE 0 END) PED_ACTION_CROSSING_WITH_SIGNAL
               ,sum(CASE WHEN vic.PED_ACTION = 'Crossing Against Signal' THEN 1 ELSE 0 END) PED_ACTION_CROSSING_AGAINST_SIGNAL
               ,sum(CASE WHEN vic.PED_ACTION in ('Crossing, No Signal, or Crosswalk', 'Crossing, No Signal, Marked Crosswalk') THEN 1 ELSE 0 END) PED_ACTION_Crossing_NO_SIGNAL
               ,sum(CASE WHEN vic.PED_ACTION not in ('Crossing With Signal', 'Crossing Against Signal', 'Crossing, No Signal, or Crosswalk', 'Crossing, No Signal, Marked Crosswalk') 
                         THEN 1 ELSE 0 END) PED_ACTION_OTHER
               ,core.ACCIDENT_DESC ACCIDENT_DESCRIPTION
FROM [FORMS].[dbo].[WC_ACCIDENT_F] core
LEFT JOIN (SELECT DISTINCT ACCIDENT_ID
                          ,PED_ACTION
                          ,PED_NONPED
                          ,PERSON_ROLE_CODE
                          ,INJ_KILLED
           FROM [FORMS].[dbo].[WC_ACCIDENT_VICTIM_F]
           WHERE PERSON_ROLE_CODE in ('Driver','Passenger','Pedestrian','In-Line Skater')) vic
ON core.integration_id = vic.accident_id
WHERE YEAR(core.ACCIDENT_DT)>2016
AND coalesce(core.VOID_STATUS_CD , 'N') ='N'
AND (core.nodeid in ({nodes})) 
AND coalesce(core.nodeid, core.lion_node_number) is not null
GROUP BY coalesce(core.nodeid, core.lion_node_number) 
        ,core.INTEGRATION_ID
        ,core.ACCIDENT_DT 
        ,left(stuff(STUFF(right('000000'+cast(core.ACCIDENT_TIME_WID AS VARCHAR),6),5,0,':'),3,0,':'), 5) 
        ,core.SRC_ADDRESS_TYPE 
        ,core.SRC_ON_STREET 
        ,core.SRC_CROSS_STREET 
        ,core.ACCIDENT_DIAGRAM
        ,core.TRAFFIC_CONTROL
        ,core.ACCIDENT_DESC 
        
        
""".format(nodes=1136)
                    )

In [5]:
data.to_excel('crash_request_output_{}.xlsx'.format(ts.replace('-','')),index=False)